# SVD++ on ML-1m

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
!wget -q --show-progress https://github.com/sparsh-ai/stanza/raw/S629908/rec/CDL/data/ml_100k_train.npy
!wget -q --show-progress https://github.com/sparsh-ai/stanza/raw/S629908/rec/CDL/data/ml_100k_test.npy

In [ ]:
train = np.load('ml_100k_train.npy')
test = np.load('ml_100k_test.npy')

train_imp = (train > 0).astype(float)
test_imp = (test > 0).astype(float)

In [ ]:
class Config:
    learning_rate = 0.01
    weight_decay = 0.1
    early_stopping_round = 0
    epochs = 100
    seed = 1995
    dim_f = 15

config = Config()

In [ ]:
class SVDpp():
    
    def __init__(self, train_exp, train_imp, test_exp, test_imp, dim_f, seed):
        self.R_tr = train_exp
        self.N_tr = train_imp
        self.R_tst = test_exp
        self.N_tst = test_imp
        
        self.dim_f = dim_f
        self.user_num = train_exp.shape[0]
        self.item_num = train_exp.shape[1]

        np.random.seed(seed)
        self.P = np.random.standard_normal((self.user_num, dim_f))
        self.Q = np.random.standard_normal((self.item_num, dim_f))
        self.Y = np.random.standard_normal((self.item_num, dim_f))

        self.B_u = np.random.standard_normal(self.user_num)
        self.B_i = np.random.standard_normal(self.item_num)
        self.mu = np.mean(train_exp[train_exp != 0])
        
        self.loss_tr = defaultdict(float)
        self.loss_tst = defaultdict(float)

    def fit(self):
        start = datetime.now()
        for epoch in range(config.epochs):
            # stochastic 
            n = 0
            for u in range(self.user_num):
                N_u = np.where(self.N_tr[u, :] != 0)[0]

                for i in range(self.item_num):
                    # rating 있는 애들만
                    if self.R_tr[u, i] != 0:                 
                        # p, q, bu, bi, y update
                        self.loss_tr[epoch] += self.gradient_descent(u, i, N_u)
                        n += 1
            
            self.loss_tr[epoch] = np.sqrt(self.loss_tr[epoch]/n )
            self.loss_tst[epoch] = self.evaluate()
            if epoch % 10 == 0 or epoch == config.epochs-1:
                print(f'EPOCH {epoch+1} : TRAINING RMSE {self.loss_tr[epoch]:.5f}, VALID RMSE {self.loss_tst[epoch]:.5f}')
        end = datetime.now()
        print(f'Training takes time {end-start}')
        
    def scoring(self, u, i, N_u):
        p = self.P[u] + np.sum(self.Y[N_u], axis=0)/np.sqrt(len(N_u))
        return self.mu + self.B_u[u] + self.B_i[i] + np.dot(p, self.Q[i].T)
    
    def gradient(self, u, i, N_u):
        loss =  self.R_tr[u, i] - self.scoring(u, i, N_u)
        added = np.sum(self.Y[N_u], axis=0)/np.sqrt(len(N_u))

        dp = loss*self.Q[i] - config.weight_decay*self.P[u]
        dq = loss*(self.P[u] + added) - config.weight_decay*self.Q[i]
        dbu = loss - config.weight_decay*self.B_u[u]
        dbi = loss - config.weight_decay*self.B_i[i]
        dyj = (loss*self.Q[i]/np.sqrt(len(N_u))).reshape(1, -1) - config.weight_decay*self.Y[N_u]
        
        return dp, dq, dbu, dbi, dyj, loss**2

    def gradient_descent(self, u, i, N_u):
        dp, dq, dbu, dbi, dyj, loss = self.gradient(u, i, N_u)
        self.P[u] = self.P[u] + config.learning_rate * dp
        self.Q[i] = self.Q[i] + config.learning_rate * dq
        self.B_u[u] = self.B_u[u] + config.learning_rate * dbu
        self.B_i[i] = self.B_i[i] + config.learning_rate * dbi
        self.Y[N_u] = self.Y[N_u] + config.learning_rate * dyj
        
        return loss

    def predict(self):
        P = np.zeros(self.P.shape)
        for u in range(self.user_num):
            N_u = np.where(self.N_tr[u, :] != 0)[0]
            P[u] = self.P[u] + np.sum(self.Y[N_u], axis=0) / np.sqrt(len(N_u))

        return self.mu + self.B_u.reshape(-1, 1) + self.B_i.reshape(1, -1) + np.dot(P, self.Q.T)

    def evaluate(self):
        pred = self.predict()
        rating_idx = self.R_tst != 0
        
        loss_pred = np.sqrt(np.mean(np.power((self.R_tst - pred)[rating_idx], 2)))

        return loss_pred

    def plot_loss(self):
        fig, ax = plt.subplots(1,1, figsize=(10, 5))
        
        ax.plot(list(self.loss_tr.keys()), list(self.loss_tr.values()), color='orange', label='train')
        ax.plot(list(self.loss_tst.keys()), list(self.loss_tst.values()), color='green', label='valid')
        plt.legend()
        plt.show()

mf = SVDpp(train, train_imp, test, test_imp, config.dim_f, config.seed)

mf.fit()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in true_divide


EPOCH 1 : TRAINING RMSE 3.03118, VALID RMSE 1.89754
EPOCH 11 : TRAINING RMSE 0.85654, VALID RMSE 0.80347
EPOCH 21 : TRAINING RMSE 0.76576, VALID RMSE 0.73089
EPOCH 31 : TRAINING RMSE 0.71779, VALID RMSE 0.69055
EPOCH 41 : TRAINING RMSE 0.68115, VALID RMSE 0.65845
EPOCH 51 : TRAINING RMSE 0.65121, VALID RMSE 0.63156
EPOCH 61 : TRAINING RMSE 0.62678, VALID RMSE 0.60926
EPOCH 71 : TRAINING RMSE 0.60692, VALID RMSE 0.59093
EPOCH 81 : TRAINING RMSE 0.59072, VALID RMSE 0.57585
EPOCH 91 : TRAINING RMSE 0.57736, VALID RMSE 0.56335
EPOCH 100 : TRAINING RMSE 0.56724, VALID RMSE 0.55383
Training takes time 0:04:19.953386


---

In [ ]:
!pip install -q watermark
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d

Author: Sparsh A.

Last updated: 2021-11-28 16:25:03

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.104+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

IPython   : 5.5.0
numpy     : 1.19.5
matplotlib: 3.2.2
pandas    : 1.1.5



---

**END**